In [13]:
# Eric's modification

import os
import torch
import numpy as np

from torch.utils.data import TensorDataset, DataLoader
from os import path

from src.models import *
from src.methods import *
from src.utils import *

data_path = os.path.join(os.getcwd(), 'project_data\\')


USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [16]:
X_test = np.load(data_path+"X_test.npy")
y_test = np.load(data_path+"y_test.npy")
person_train_valid = np.load(data_path+"person_train_valid.npy")
X_train_valid = np.load(data_path+"X_train_valid.npy")
y_train_valid = np.load(data_path+"y_train_valid.npy")
person_test = np.load(data_path+"person_test.npy")
#:\Users\pluto\Desktop\2024_winter\ECE_147\final_project\ECE247-Group-Project\src\project_data\y_test.npy

In [17]:
y_train_valid-=769
y_test-=769

In [18]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [19]:
batch_size = 64

In [20]:
X_train_valid = torch.tensor(X_train_valid, dtype=torch.float)
y_train_valid = torch.tensor(y_train_valid, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)

train_valid_dataset = TensorDataset(X_train_valid, y_train_valid)
test_dataset = TensorDataset(X_test, y_test)

num_train = int(0.8 * len(train_valid_dataset))
num_valid = len(train_valid_dataset) - num_train
train_dataset, valid_dataset = torch.utils.data.random_split(train_valid_dataset, [num_train, num_valid])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

train(model, train_loader, valid_loader, criterion, optimizer, device=device, epochs=50)

test_accuracy = evaluate(model, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 20%|██        | 10/50 [00:19<00:50,  1.26s/it]

Epoch [10/50], Loss: 3.8457, Validation Accuracy: 32.15%


 40%|████      | 20/50 [00:30<00:35,  1.19s/it]

Epoch [20/50], Loss: 2.2498, Validation Accuracy: 39.72%


 60%|██████    | 30/50 [00:42<00:23,  1.19s/it]

Epoch [30/50], Loss: 1.3907, Validation Accuracy: 47.28%


 80%|████████  | 40/50 [00:54<00:11,  1.20s/it]

Epoch [40/50], Loss: 0.8834, Validation Accuracy: 48.70%


100%|██████████| 50/50 [01:06<00:00,  1.32s/it]

Epoch [50/50], Loss: 0.5142, Validation Accuracy: 46.34%
Test Accuracy: 47.86%


In [23]:
lstm = LSTMModel(input_size=1000)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

lstm = lstm.to(device)
train(lstm, train_loader, valid_loader, criterion, optimizer, device=device)

test_accuracy = evaluate(lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 50%|█████     | 10/20 [00:03<00:03,  2.78it/s]

Epoch [10/20], Loss: 0.1380, Validation Accuracy: 30.73%


100%|██████████| 20/20 [00:07<00:00,  2.73it/s]

Epoch [20/20], Loss: 0.0828, Validation Accuracy: 28.37%
Test Accuracy: 27.99%


In [24]:
cnn_lstm = CNN_LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

cnn_lstm = cnn_lstm.to(device)
train(cnn_lstm, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100)

test_accuracy = evaluate(cnn_lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

  0%|          | 0/100 [00:00<?, ?it/s]

 10%|█         | 10/100 [00:11<01:45,  1.17s/it]

Epoch [10/100], Loss: 1.0818, Validation Accuracy: 43.74%


 20%|██        | 20/100 [00:23<01:35,  1.19s/it]

Epoch [20/100], Loss: 0.4310, Validation Accuracy: 54.37%


 30%|███       | 30/100 [00:35<01:23,  1.19s/it]

Epoch [30/100], Loss: 0.2250, Validation Accuracy: 56.97%


 37%|███▋      | 37/100 [00:44<01:15,  1.20s/it]

Early stopping!
Test Accuracy: 55.98%
